My plan is to plot the average lap time of everybody in every race.  Since the lap times for each race are different, we need to normalise by the course used on the day.  Every race needs its own baseline.  I will use the fastest lap of the day as a baseline for each course.

I need to load data from multiple races and the hunter races (hmtba) are split into all the separate grades.

All the hunter files are in the same format, so I can load them all up and concatenate them.  Empty cells come in as a space, so I convert that to nan.  I then convert all the strings that represent times into actual times so the numerical operations can work on them

Finally I add calculated columns to give each rider their own average and give everyone in the whole data frame the same minimum lap time.

In [5]:
import pandas as pd
import numpy as np

a_grade = pd.read_csv('data/hmtba/2022-XC-Round-3-AGrade.csv')[["name","lap1", "lap2", "lap3", "lap4", "lap5"]]
b_grade = pd.read_csv('data/hmtba/2022-XC-Round-3-BGrade.csv')[["name","lap1", "lap2", "lap3", "lap4", "lap5"]]
c_grade = pd.read_csv('data/hmtba/2022-XC-Round-3-CGrade.csv')[["name","lap1", "lap2", "lap3", "lap4", "lap5"]]
d_grade = pd.read_csv('data/hmtba/2022-XC-Round-3-DGrade.csv')[["name","lap1", "lap2", "lap3", "lap4", "lap5"]]
hmtba_xc3 = pd.concat([a_grade, b_grade, c_grade, d_grade])
hmtba_xc3 = hmtba_xc3.replace('^\s*$', np.nan, regex=True)
hmtba_xc3["lap1"] = pd.to_timedelta(hmtba_xc3["lap1"])
hmtba_xc3["lap2"] = pd.to_timedelta(hmtba_xc3["lap2"])
hmtba_xc3["lap3"] = pd.to_timedelta(hmtba_xc3["lap3"])
hmtba_xc3["lap4"] = pd.to_timedelta(hmtba_xc3["lap4"])
hmtba_xc3["lap5"] = pd.to_timedelta(hmtba_xc3["lap5"])
hmtba_xc3["avglap"] = hmtba_xc3[["lap1", "lap2", "lap3", "lap4", "lap5"]].mean(axis=1)
hmtba_xc3["minlap"] = hmtba_xc3[["lap1", "lap2", "lap3", "lap4", "lap5"]].min().min()


The process is similar for the one race I have from western sydney (wsmtb).  All the grades are in the one file, so it is easier in that respect.  However, the western sydney data contains comment rows, emtpy rows, and rows for people who did a shorter race.  They all need to be removed.

In [6]:
wsmtb_xc3 = pd.read_csv('data/wsmtb/2022-XC-Round-3-Results.csv', skiprows=2)[["Name","Lap1", "Lap2", "Lap3", "Lap4", "Lap5"]]
only_riders = wsmtb_xc3.index.notnull()
wsmtb_xc3 = wsmtb_xc3[only_riders]
# filter junk rows
wsmtb_xc3 = wsmtb_xc3[wsmtb_xc3["Lap1"] != "Lap1"]
# kill centiseconds
wsmtb_xc3 = wsmtb_xc3.replace("^(.*\d),\d$", "\\1", regex=True)
# to timedelta and rename
wsmtb_xc3["lap1"] = pd.to_timedelta(wsmtb_xc3["Lap1"])
wsmtb_xc3["lap2"] = pd.to_timedelta(wsmtb_xc3["Lap2"])
wsmtb_xc3["lap3"] = pd.to_timedelta(wsmtb_xc3["Lap3"])
wsmtb_xc3["lap4"] = pd.to_timedelta(wsmtb_xc3["Lap4"])
wsmtb_xc3["lap5"] = pd.to_timedelta(wsmtb_xc3["Lap5"])
wsmtb_xc3["name"] = wsmtb_xc3["Name"]
wsmtb_xc3 = wsmtb_xc3[["name","lap1", "lap2", "lap3", "lap4", "lap5"]]
# exclude kids laps
wsmtb_xc3 = wsmtb_xc3[wsmtb_xc3["lap1"] > pd.Timedelta(11, "minutes")]
wsmtb_xc3["avglap"] = wsmtb_xc3[["lap1", "lap2", "lap3", "lap4", "lap5"]].mean(axis=1)
wsmtb_xc3["minlap"] = wsmtb_xc3[["lap1", "lap2", "lap3", "lap4", "lap5"]].min().min()


Now I just compbine the two data frames.  They have the same columns, so `concat` can do the job for me.

In [7]:
races = pd.concat([wsmtb_xc3, hmtba_xc3])
races


,name,lap1,lap2,lap3,lap4,lap5,avglap,minlap
1,CAMERON WINFIELD,0 days 00:11:19,0 days 00:11:59,0 days 00:12:04,0 days 00:11:05,0 days 00:11:28,0 days 00:11:35,0 days 00:11:05
2,BLAKE SUTCLIFFE,0 days 00:11:20,0 days 00:11:59,0 days 00:12:03,0 days 00:11:26,0 days 00:11:40,0 days 00:11:41.600000,0 days 00:11:05
3,LUKE BRAME,0 days 00:11:19,0 days 00:11:59,0 days 00:12:04,0 days 00:12:00,0 days 00:11:29,0 days 00:11:46.200000,0 days 00:11:05
4,DAVID WINFIELD,0 days 00:11:50,0 days 00:11:56,0 days 00:11:59,0 days 00:11:59,0 days 00:12:17,0 days 00:12:00.200000,0 days 00:11:05
5,SIMON EBBELING,0 days 00:12:21,0 days 00:12:34,0 days 00:12:45,0 days 00:12:58,0 days 00:13:03,0 days 00:12:44.200000,0 days 00:11:05
...,...,...,...,...,...,...,...,...
3,Peter Riccio,0 days 00:22:48.100000,0 days 00:22:49.100000,NaT,NaT,NaT,0 days 00:22:48.600000,0 days 00:16:23.500000
4,Simon McElduff,0 days 00:22:36.500000,0 days 00:23:45.100000,NaT,NaT,NaT,0 days 00:23:10.800000,0 days 00:16:23.500000
5,Hamish Rae,0 days 00:23:54.600000,0 days 00:26:33.700000,NaT,NaT,NaT,0 days 00:25:14.150000,0 days 00:16:23.500000
6,Peter Woodward,0 days 00:26:57.700000,NaT,NaT,NaT,NaT,0 days 00:26:57.700000,0 days 00:16:23.500000
